In [ ]:
def clean_and_process(x):
    ## Round 2
    ## Standardize Headers
    #Drop the first "Unnamed: 0" column
    if 'Unnamed: 0' in x.columns:
        x = x.drop(['Unnamed: 0'], axis=1)
    else:
        x = x
    #Headers all on lower case and replace the space by "_"
    header = []
    for item in x.columns:
        header.append(item.lower().replace(' ', '_'))
    x.columns = header
    #Headers without space
    x = x.rename(columns={'EmploymentStatus':'Employment Status'})
    
    ## Check for duplicates
    x = x.drop_duplicates()
    
    ## taking care of the NAN-Values:
    #droping the NAN-Values for 'state' and 'response'
    x = x[x['state'].isna()==False]
    #droping the NAN-Values for 'months_since_last_claim' and 'number_of_open_complaints'
    x = x[x['months_since_last_claim'].isna()==False]
    #fill the 'vehicle_class' with the mean value
    x['vehicle_class'] = x['vehicle_class'].fillna(x['vehicle_class'].value_counts(dropna=True).index[0])
    #fill the 'vehicle_size' with the mean value
    x['vehicle_size'] = x['vehicle_size'].fillna(x['vehicle_size'].value_counts(dropna=True).index[0])
    #fill the NAN of vehicle_type with 'not A', because 50%
    x['vehicle_type'] = x['vehicle_type'].fillna('not A')
    #
    x['effective_to_date'] = pd.to_datetime(x['effective_to_date'], errors='coerce')
    x['effective_to_date_month'] = x['effective_to_date'].dt.month
    
    ## Round 3
    # describing the numerical columns
    summary_numerical = x.describe().T 

    # additionnaly we add the range and the iqr
    summary_numerical['iqr'] = summary_numerical['75%']-summary_numerical['25%']
    summary_numerical['range'] = summary_numerical['max']-summary_numerical['min']
        
    # categorical
    summary_objects = x.describe(include=[np.object]).T
    
    #DataFrame info
    info_data = x.info()
    display(info_data)
    
    #Plottings --> no use in cleaning and processing the data, so not coded in the function.
    #Round 4 was correlation with the numericals, this is done again in Round 5.
    
    ## Round 5
    #X-y split
    y = x['total_claim_amount']
    X = x.drop(['total_claim_amount', 'customer', 'effective_to_date', 'effective_to_date_month', 'numerical_response','claims_bin','income_bin'], axis=1)
    
    X_num = X.select_dtypes(np.number)
    X_cat = X.select_dtypes(object)
    
    # Normalizing data - MinMaxScaler:
    transformer = MinMaxScaler().fit(X_num)
    X_num_normalized = transformer.transform(X_num)
    X_num_normalized = pd.DataFrame(X_num_normalized, columns=X_num.columns)
    
#     # Standardization or Z-Score Normalization
#     transformer = StandardScaler().fit(X_num)
#     X_num_normalized = transformer.transform(X_num)
#     X_num_normalized = pd.DataFrame(X_num_normalized, columns=X_num.columns)
    
    #Categorical Variables
    encoder = OneHotEncoder(drop='first').fit(X_cat) # the first one is the most frequent one.
    encoded = encoder.transform(X_cat).toarray()
    cols = encoder.get_feature_names(input_features=X_cat.columns)
    onehot_encoded = pd.DataFrame(encoded, columns=cols)
    
    #Concatenating:
    X = pd.concat([X_num_normalized, onehot_encoded], axis=1)
    
    #Correlation
    correlations_matrix = round(numerical.corr(),2)
    
    return X  